In [ ]:
MAX_LEN = 256
bs = 32

In [ ]:
import pandas as pd
train = '/content/NER_Irish_train.conll'
val = '/content/NER_Irish_validation.conll'
test = '/content/NER_Irish_test.conll'

In [ ]:
!pip install transformers
!pip install tokenizers

In [ ]:
def read_conll_file(file_path):
    data = []
    current_sentence = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line.startswith('-DOCSTART-'):
                continue
            if line:
                parts = line.split()
                word = parts[0]
                ner_label = parts[-1]
                current_sentence.append((word, ner_label))
            else:
                if current_sentence:
                    data.append(current_sentence)
                    current_sentence = []
    if current_sentence:
        data.append(current_sentence)
    return data

train_data = read_conll_file(train)
val_data = read_conll_file(val)
test_data = read_conll_file(test)

In [ ]:
print(train_data)

[[('\ufeffDáil', 'B-ORG'), ('Éireann', 'I-ORG'), ('06', 'O'), ('/', 'O'), ('07', 'O'), ('/', 'O'), ('2023', 'O')], [('Baineann', 'O'), ('mo', 'O'), ('cheist', 'O'), ('le', 'O'), ('cúrsaí', 'O'), ('tithíochta', 'O'), ('i', 'O'), ('nGaillimh', 'B-LOC'), ('-', 'O'), ('tá', 'O'), ('sé', 'O'), ('níos', 'O'), ('cirte', 'O'), ('easpa', 'O'), ('tithíochta', 'O'), ('i', 'O'), ('nGaillimh', 'B-LOC'), ('a', 'O'), ('rá', 'O'), ('-', 'O'), ('agus', 'O'), ('an', 'O'), ('tascfhórsa', 'O'), ('a', 'O'), ('bunaíodh', 'O'), ('breis', 'O'), ('agus', 'O'), ('ceithre', 'O'), ('bliana', 'O'), ('ó', 'O'), ('shin', 'O')], [('Ar', 'O'), ('dheis', 'O'), ('Dé', 'B-PER'), ('go', 'O'), ('raibh', 'O'), ('a', 'O'), ('anam', 'O')], [('Táim', 'O'), ('os', 'O'), ('comhair', 'O'), ('Dail', 'B-ORG'), ('Éireann', 'I-ORG')], [('Ba', 'O'), ('mhaith', 'O'), ('liom', 'O'), ('tréaslú', 'O'), ('leis', 'O'), ('an', 'O'), ('Teachta', 'O'), ('Andrews', 'B-PER'), ('as', 'O'), ('an', 'O'), ('obair', 'O'), ('atá', 'O'), ('déanta', 'O'

In [ ]:
print(val_data)

[[('Crothnófar', 'O'), ('Pól', 'B-PER'), ('nó', 'O'), ('ba', 'O'), ('úrlabhraí', 'O'), ('maith', 'O'), ('é', 'O'), ('ar', 'O'), ('Raidió', 'B-ORG'), ('na', 'I-ORG'), ('Gaeltachta', 'I-ORG'), ('agus', 'O'), ('na', 'O'), ('meáin', 'O'), ('eile', 'O'), ('ag', 'O'), ('cosaint', 'O'), ('na', 'O'), ('n-oifigí', 'O'), ('poist', 'O'), ('tuaithe', 'O'), (',', 'O'), ('gné', 'O'), ('am-tábhachtach', 'O'), ('de', 'O'), ('shaol', 'O'), ('sóisialta', 'O'), ('na', 'O'), ('ndaoine', 'O'), ('.', 'O')], [('Dá', 'O'), ('mba', 'O'), (',', 'O'), ('mar', 'O'), ('sin', 'O'), (',', 'O'), ('nach', 'O'), ('bhfaca', 'O'), ('an', 'O'), ('tSiúr', 'O'), ('Concepta', 'B-PER'), ('an', 'O'), ('tréadaí', 'O'), ('Eoin', 'B-PER'), ('Mac', 'I-PER'), ('Diarmada', 'I-PER'), ('ag', 'O'), ('bailiú', 'O'), ('leis', 'O'), ('ag', 'O'), ('an', 'O'), ('mbreacsholas', 'O'), (',', 'O'), ('níorbh', 'O'), ('aistí', 'O'), ('rud', 'O'), ('riamh', 'O'), ('ná', 'O'), ('an', 'O'), ('radharc', 'O'), ('a', 'O'), ('bhí', 'O'), ('le', 'O'), ('

In [ ]:
print(test_data)

[[('Mar', 'O'), ('a', 'O'), ('tchítear', 'O'), ('do', 'O'), ('Sheosamh', 'B-PER'), ('Mac', 'I-PER'), ('Grianna', 'I-PER'), ('é', 'O'), ('caithfidh', 'O'), ('an', 'O'), ('t-ealaíontóir', 'O'), ('an', 'O'), ('solas', 'O'), ('a', 'O'), ('thabhairt', 'O'), ('don', 'O'), ('saol', 'O'), ('agus', 'O'), ('diúltú', 'O'), ('do', 'O'), ('chathú', 'O'), ('sin', 'O'), ('na', 'O'), ('truaillíochta', 'O'), ('a', 'O'), ('chuireann', 'O'), ('an', 'O'), ('saol', 'O'), ('ina', 'O'), ('chosán', 'O'), ('.', 'O')], [('Dhiúltaigh', 'O'), ('Johnathan', 'B-PER'), ('Cape', 'I-PER'), ('agus', 'O'), ('cúpla', 'O'), ('foilsitheoir', 'O'), ('eile', 'O'), ('an', 'O'), ('leabhar', 'O'), ('a', 'O'), ('fhoilsiú', 'O'), ('agus', 'O'), ('bhí', 'O'), ('Naomi', 'B-PER'), ('le', 'O'), ('ceangal', 'O'), ('go', 'O'), ('raibh', 'O'), ('srian', 'O'), ('a', 'O'), ('chur', 'O'), ('ar', 'O'), ('a', 'O'), ('cuid', 'O'), ('tuairimíochta', 'O'), ('.', 'O')], [('Grianghraif', 'O'), ('le', 'O'), ('Maidhc', 'B-PER'), ('Ó', 'I-PER'), ('S

In [ ]:
import torch
from transformers import AutoTokenizer

# Enocde the labels
def encode_labels(sentence, tokenizer, label_map):
    tokens = []
    encoded_labels = []
    word_indices = []

    for word, label in sentence:
        word_tokens = tokenizer.tokenize(word)
        word_indices.append(len(tokens))  # Track the index of the original word
        tokens.extend(word_tokens)
        # Assign label to each sub-token
        for _ in word_tokens:
            encoded_labels.append(label_map[label])

    return tokens, encoded_labels, word_indices

def prepare_bert_inputs(data):
    tokenizer = AutoTokenizer.from_pretrained('DCU-NLP/bert-base-irish-cased-v1')
    bert_inputs = []
    max_seq_length = 256  # Defining the maximum sequence length

    # Dictionary of labels mapped to integer representations
    label_map = {
        'O': 0,
        'B-PER': 1,
        'I-PER': 2,
        'B-LOC': 3,
        'I-LOC': 4,
        'B-ORG': 5,
        'I-ORG': 6
    }

    for sentence in data:
        tokens, encoded_labels, word_indices = encode_labels(sentence, tokenizer, label_map)

        # Padding the sequences
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        padding_length = max_seq_length - len(tokens)
        tokens += [tokenizer.pad_token] * padding_length

        # 0 corresponds to 'O' tag
        encoded_labels = [0] + encoded_labels + [0]

        # Adjust encoded_labels based on original word indices
        adjusted_labels = [encoded_labels[idx] for idx in word_indices]
        adjusted_labels += [0] * (max_seq_length - len(adjusted_labels))

        # Converting the tokens to input IDs and creating attention mask
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        # Padding the attention_mask
        padding_length = max_seq_length - len(attention_mask)
        attention_mask += [0] * padding_length

        # Converting adjusted_labels to tensor
        adjusted_labels = torch.tensor(adjusted_labels, dtype=torch.long)

        # Appending the data to bert_inputs in a dict format
        bert_inputs.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': adjusted_labels
        })

    return bert_inputs

In [ ]:
train_inputs = prepare_bert_inputs(train_data)

In [ ]:
print(train_data[12])
print(train_inputs[12])

[('Ní', 'O'), ('féidir', 'O'), ('linn', 'O'), ('muinín', 'O'), ('a', 'O'), ('bheith', 'O'), ('againn', 'O'), ('as', 'O'), ('an', 'O'), ('eolas', 'O'), ('a', 'O'), ('thagann', 'O'), ('ó', 'O'), ('mbord', 'O'), ('feidhmiúcháin', 'O'), ('Nike', 'B-ORG'), ('agus', 'O'), ('tá', 'O'), ('feidhmeannaigh', 'O'), ('an', 'O'), ('bhoird', 'O'), ('ag', 'O'), ('caitheamh', 'O'), ('go', 'O'), ('holc', 'O'), ('leis', 'O'), ('an', 'O'), ('bpobal', 'O'), ('.', 'O')]
{'input_ids': [102, 421, 450, 739, 5735, 105, 378, 1152, 343, 115, 1134, 105, 2492, 228, 8314, 8829, 22096, 138, 308, 26003, 115, 10440, 124, 4107, 173, 17568, 239, 115, 4447, 118, 103, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Testing Augmentation

In [ ]:
train_data[0:10]

[[('\ufeffDáil', 'B-ORG'),
  ('Éireann', 'I-ORG'),
  ('06', 'O'),
  ('/', 'O'),
  ('07', 'O'),
  ('/', 'O'),
  ('2023', 'O')],
 [('Baineann', 'O'),
  ('mo', 'O'),
  ('cheist', 'O'),
  ('le', 'O'),
  ('cúrsaí', 'O'),
  ('tithíochta', 'O'),
  ('i', 'O'),
  ('nGaillimh', 'B-LOC'),
  ('-', 'O'),
  ('tá', 'O'),
  ('sé', 'O'),
  ('níos', 'O'),
  ('cirte', 'O'),
  ('easpa', 'O'),
  ('tithíochta', 'O'),
  ('i', 'O'),
  ('nGaillimh', 'B-LOC'),
  ('a', 'O'),
  ('rá', 'O'),
  ('-', 'O'),
  ('agus', 'O'),
  ('an', 'O'),
  ('tascfhórsa', 'O'),
  ('a', 'O'),
  ('bunaíodh', 'O'),
  ('breis', 'O'),
  ('agus', 'O'),
  ('ceithre', 'O'),
  ('bliana', 'O'),
  ('ó', 'O'),
  ('shin', 'O')],
 [('Ar', 'O'),
  ('dheis', 'O'),
  ('Dé', 'B-PER'),
  ('go', 'O'),
  ('raibh', 'O'),
  ('a', 'O'),
  ('anam', 'O')],
 [('Táim', 'O'),
  ('os', 'O'),
  ('comhair', 'O'),
  ('Dail', 'B-ORG'),
  ('Éireann', 'I-ORG')],
 [('Ba', 'O'),
  ('mhaith', 'O'),
  ('liom', 'O'),
  ('tréaslú', 'O'),
  ('leis', 'O'),
  ('an', 'O'),
  ('

In [ ]:
import random

def augment_sentence_with_entities_v3(sentence, entity_pool, num_entities=1):
    augmented_sentence = sentence[:]  # Make a copy of the sentence to preserve the original data.

    # Determine insertion positions excluding the last position unless it's 'O'.
    insert_positions = [i for i, (_, label) in enumerate(sentence[:-1]) if label == 'O' and sentence[i+1][1] == 'O']
    if sentence[-1][1] == 'O':
        insert_positions.append(len(sentence)-1)  # Include the last position if it's 'O'.

    for _ in range(num_entities):
        if not insert_positions:
            break  # Exit if no suitable position is found.

        insert_position = random.choice(insert_positions)
        # Select a random 'B-' tag entity to insert.
        b_tag_entities = {k: v for k, v in entity_pool.items() if k.startswith('B-')}
        entity_label = random.choice(list(b_tag_entities.keys()))
        entity_to_insert = random.choice(b_tag_entities[entity_label])
        entity_tokens = entity_to_insert.split()

        # Insert the 'B-' entity at the chosen position.
        for token in entity_tokens:
            augmented_sentence.insert(insert_position + 1, (token, entity_label))
            insert_position += 1  # Update the position for the next token insertion.

        i_tag_tokens = []  # Initialise the list to avoid the NameError.
        # Optionally insert 'I-' tags following the 'B-' tag, ensuring logical consistency.
        i_tag_label = 'I' + entity_label[1:]  # Convert 'B-' tag to 'I-' tag.
        if i_tag_label in entity_pool and entity_pool[i_tag_label]:
            i_tag_entity = random.choice(entity_pool[i_tag_label])
            i_tag_tokens = i_tag_entity.split()
            for token in i_tag_tokens:
                # Ensure the insertion follows logical consistency rules.
                if insert_position < len(augmented_sentence) - 1 and (augmented_sentence[insert_position + 1][1] == 'O' or augmented_sentence[insert_position + 1][1].startswith('I-')):
                    augmented_sentence.insert(insert_position + 1, (token, i_tag_label))
                    insert_position += 1

        # Update insert positions to avoid clustering inserted entities.
        insert_positions = [pos for pos in insert_positions if pos < insert_position or pos > insert_position + len(entity_tokens) + len(i_tag_tokens)]

    return augmented_sentence



In [ ]:
entity_pool = {}
i = 0
while i < len(train_data):
  for word, label in train_data[i]:
    if label != 'O':
        if label not in entity_pool:
            entity_pool[label] = []
        if word not in entity_pool[label]:
            entity_pool[label].append(word)
  i += 1  # Increment i inside the while loop

print(entity_pool)
print(len(entity_pool['B-PER']))
print(len(entity_pool['I-PER']))
print(len(entity_pool['B-ORG']))
print(len(entity_pool['I-ORG']))
print(len(entity_pool['B-LOC']))
print(len(entity_pool['I-LOC']))



{'B-ORG': ['\ufeffDáil', 'Dail', 'Dáil', 'Fís', 'RTÉ', 'TG4', 'Údarás', 'Nike', 'an', 'Sky', 'Dála', 'Sinn', 'An', 'HSE', 'Raidió', 'Sláintecare', 'Chumann', 'gComhaontas', 'Fianna', 'Fine', 'd’Fhianna', 'Teach', 'Roinne', 'Cúla4', 'd’Údarás', 'DEIS', 'Chonradh', 'Massie', 'New', 'John', 'bhForas', '“Irish', 'VirginMedia', 'S4C', 'CLG', 'gCumann', 'Fórsa', 'SIPTU', 'INMO', 'Enable', 'Tusla', 'FSS', 'Thithe', 'Foras', 'gComhairle', 'CATT', 'Conradh', 'Glór', 'Gael', 'Údaras', 'gComhchoiste', 'Comhchoiste', 'Comhairle', 'Chomhairle', 'Rannóg', 'Cluichí', 'Cumann', 'Citylink', 'Coláiste', 'Seanad', 'Choláiste', 'Rialtas', 'Aontas', 'Gharda', 'Banc', 'Fhine', 'Fhianna', 'hArd-Chúirte', 'Oireachtas', 'Oireachtais', 'Grant', 'Choiste', 'Chomhchoiste', 'Roinn', "'Seirbhísí", 'Comhdháil', 'Cúldoras', 'Acadamh', 'FBI', 'Chlub', 'Tia', 'Oifig', 'INTRA', 'gColáiste', 'College', 'hÉireannaigh', 'hArm', 'BBC', 'Globalise', 'gCuntas', 'Thoghroinn', 'Náisiún', 'Faisean', 'Éire', 'Scoil', 'ECU', 'ANOI

In [ ]:
import random

def replace_and_adjust_entities(sentence, words_to_replace, entity_pool):
    updated_sentence = []
    for i, (word, label) in enumerate(sentence):
        if word in words_to_replace:
            # Choose a random 'B-PER' entity to replace
            b_per_entity = random.choice(entity_pool['B-PER']).split()
            for j, entity_word in enumerate(b_per_entity):
                # Use 'B-PER' for the first token, 'I-PER' for subsequent tokens
                new_label = 'I-PER' if j > 0 else 'B-PER'
                updated_sentence.append((entity_word, new_label))
        else:
            updated_sentence.append((word, label))

    # Adjust consecutive entities
    adjusted_sentence = []
    previous_label = None
    for word, label in updated_sentence:
        if label == 'B-PER' and previous_label == 'B-PER':
            adjusted_label = 'I-PER'
        else:
            adjusted_label = label
        adjusted_sentence.append((word, adjusted_label))
        previous_label = label

    return adjusted_sentence

# Simple example
sentence = [('Sí', 'O'), ('Sé', 'O'), ('to', 'O'), ('the', 'O'), ('store', 'O'), ('and', 'O'), ('Sé', 'O'), ('followed', 'O')]
words_to_replace = ['Sí', 'Sé']
updated_and_adjusted_sentence = replace_and_adjust_entities(sentence, words_to_replace, entity_pool)
print(updated_and_adjusted_sentence)


[('Julian', 'B-PER'), ('Mairéad', 'I-PER'), ('to', 'O'), ('the', 'O'), ('store', 'O'), ('and', 'O'), ('Concubhar', 'B-PER'), ('followed', 'O')]


In [ ]:
def adjust_consecutive_entities(sentence):
    adjusted_sentence = []
    previous_label = None  # Keep track of the previous label

    for word, label in sentence:
        if label == 'B-PER' and previous_label == 'B-PER':
            # Change to 'I-PER' if the previous label was also 'B-PER'
            adjusted_label = 'I-PER'
        else:
            adjusted_label = label

        adjusted_sentence.append((word, adjusted_label))
        previous_label = label  # Update the previous label for the next iteration

    return adjusted_sentence

# Example
sentence = [
    ('Micheal', 'B-PER'), ('Briain', 'B-PER'), ('to', 'O'), ('the', 'O'),
    ('store', 'O'), ('and', 'O'), ('Ard-Reachtaire', 'B-PER'), ('followed', 'O')
]
adjusted_sentence = adjust_consecutive_entities(sentence)
print(adjusted_sentence)


[('Micheal', 'B-PER'), ('Briain', 'I-PER'), ('to', 'O'), ('the', 'O'), ('store', 'O'), ('and', 'O'), ('Ard-Reachtaire', 'B-PER'), ('followed', 'O')]


In [ ]:
augmented_train_data_v3 = [augment_sentence_with_entities_v3(sentence, entity_pool, num_entities=1) for sentence in train_data]
#train_inputs_v2 = prepare_bert_inputs(augmented_train_data_v3, tokenizer, label_map)

print(train_data[35])
print(augmented_train_data_v3[35])

[('Tagann', 'O'), ('an', 'O'), ('chéad', 'O'), ('cheist', 'O'), ('ón', 'O'), ('Teachta', 'O'), ('Ó', 'B-PER'), ('Cuív', 'I-PER'), ('.', 'O')]
[('Tagann', 'O'), ('an', 'O'), ('chéad', 'O'), ('cheist', 'O'), ('Amharclann', 'B-LOC'), ('Éadaigh', 'I-LOC'), ('ón', 'O'), ('Teachta', 'O'), ('Ó', 'B-PER'), ('Cuív', 'I-PER'), ('.', 'O')]


# Testing on Data

In [ ]:
train_data[0]

[('\ufeffDáil', 'B-ORG'),
 ('Éireann', 'I-ORG'),
 ('06', 'O'),
 ('/', 'O'),
 ('07', 'O'),
 ('/', 'O'),
 ('2023', 'O')]

In [ ]:
augmented_train_data_v3 = [augment_sentence_with_entities_v3(sentence, entity_pool, num_entities=1) for sentence in train_data]
train_inputs = prepare_bert_inputs(augmented_train_data_v3)
train_data = augmented_train_data_v3

In [ ]:
val_inputs = prepare_bert_inputs(val_data)
test_inputs = prepare_bert_inputs(test_data)

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments
import torch

model_name = 'DCU-NLP/bert-base-irish-cased-v1'
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=7)

# Load the tokeniser
tokenizer = AutoTokenizer.from_pretrained(model_name)
torch.cuda.empty_cache()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
# Checking for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Moving the model to device to use GPU
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DCU-NLP/bert-base-irish-cased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Defining compute_metrics function for the evaluation metrics I want to see
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels.flatten(), preds.flatten())
    report = classification_report(labels.flatten(), preds.flatten(), zero_division=0)
    return {
        'accuracy': acc,
        'classification_report': report
    }

 # Defining the training arguments to be used for the Trainer
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",evaluation_strategy="epoch")

from transformers import Trainer

# Passing the model to be trained and the data for training and evaluation along with the metrics defined before

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_inputs,
    eval_dataset=val_inputs,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertConfig

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
train_sent = []

for lst in train_data:
  sent = []
  for words in lst:
    sent.append(words[0])
  train_sent.append(sent)

val_sent = []

for lst in val_data:
  sent = []
  for words in lst:
    sent.append(words[0])
  val_sent.append(sent)

test_sent = []

for lst in test_data:
  sent = []
  for words in lst:
    sent.append(words[0])
  test_sent.append(sent)

In [ ]:
train_labels = []

for lst in train_data:
  lab = []
  for words in lst:
    lab.append(words[1])
  train_labels.append(lab)

val_labels = []

for lst in val_data:
  lab = []
  for words in lst:
    lab.append(words[1])
  val_labels.append(lab)

test_labels = []

for lst in test_data:
  lab = []
  for words in lst:
    lab.append(words[1])
  test_labels.append(lab)

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenise the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenised word to the final tokenised word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(train_sent, train_labels)
]

val_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(val_sent, val_labels)
]

test_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(test_sent, test_labels)
]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

val_tokenized_texts = [token_label_pair[0] for token_label_pair in val_tokenized_texts_and_labels]
val_labels = [token_label_pair[1] for token_label_pair in val_tokenized_texts_and_labels]

test_tokenized_texts = [token_label_pair[0] for token_label_pair in test_tokenized_texts_and_labels]
test_labels = [token_label_pair[1] for token_label_pair in test_tokenized_texts_and_labels]

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

val_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in val_tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in test_tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
tag_values = [
        'O',
        'B-PER',
        'I-PER',
        'B-LOC',
        'I-LOC',
        'B-ORG',
        'I-ORG'
]

tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

val_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in val_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

test_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in test_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

val_attention_masks = [[float(i != 0.0) for i in ii] for ii in val_input_ids]

test_attention_masks = [[float(i != 0.0) for i in ii] for ii in test_input_ids]

In [ ]:
tr_inputs = torch.tensor(input_ids)
tr_tags = torch.tensor(tags)
tr_masks = torch.tensor(attention_masks)

vl_inputs = torch.tensor(val_input_ids)
vl_tags = torch.tensor(val_tags)
vl_masks = torch.tensor(val_attention_masks)

tst_inputs = torch.tensor(test_input_ids)
tst_tags = torch.tensor(test_tags)
tst_masks = torch.tensor(test_attention_masks)

In [ ]:
tr_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(tr_data)
train_dataloader = DataLoader(tr_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(vl_inputs, vl_masks, vl_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

tst_data = TensorDataset(tst_inputs, tst_masks, tst_tags)
tst_sampler = SequentialSampler(tst_data)
tst_dataloader = DataLoader(tst_data, sampler=tst_sampler, batch_size=bs)

In [ ]:
from transformers import BertForTokenClassification, AdamW, BertModel, BertConfig

In [ ]:
config = BertConfig.from_pretrained('DCU-NLP/bert-base-irish-cased-v1', output_hidden_states=True)
config.max_position_embeddings = 512

bert_model = BertModel.from_pretrained(
                        'DCU-NLP/bert-base-irish-cased-v1',
                        config=config,
                        add_pooling_layer=False
)

In [ ]:
import torch.nn as nn

In [ ]:
!pip install pytorch-crf
from torchcrf import CRF

In [ ]:
class BERT_CRF(nn.Module):
    def __init__(self, bert_model, num_labels):
        super(BERT_CRF, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.25)
        # 4 last of layer
        self.classifier = nn.Linear(4*768, num_labels)
        self.crf = CRF(num_labels, batch_first = True)

    def forward_custom(self, b_input_ids, b_input_mask,  b_labels=None, token_type_ids=None):
        outputs = self.bert(b_input_ids, attention_mask=b_input_mask)
        sequence_output = torch.cat((outputs[1][-1], outputs[1][-2], outputs[1][-3], outputs[1][-4]),-1)
        sequence_output = self.dropout(sequence_output)

        emission = self.classifier(sequence_output) # [32,256,17]

        if b_labels is not None:
            loss = -self.crf(log_soft(emission, 2), b_labels, mask=b_input_mask.type(torch.uint8), reduction='mean')
            prediction = self.crf.decode(emission, mask=b_input_mask.type(torch.uint8))
            return [loss, prediction]

        else:
            prediction = self.crf.decode(emission, mask=b_input_mask.type(torch.uint8))
            return prediction

In [ ]:
model = BERT_CRF(bert_model, num_labels=len(tag2idx))
model.to(device)

BERT_CRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
cnt = -1
num_layer = 197 #?????
for param in model.named_parameters():
    cnt += 1
    if cnt>=num_layer:
        param[1].requires_grad = True
    else:
        param[1].requires_grad = True
    print(cnt,param[0],'\t',param[1].requires_grad)


FINETUNING = True
if FINETUNING:
    param_optimizer1 = list(model.named_parameters())[:num_layer]
    param_optimizer2 = list(model.named_parameters())[num_layer:num_layer+2]
    param_optimizer3 = list(model.named_parameters())[num_layer+2:]
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer1 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 1e-5},
        {'params': [p for n, p in param_optimizer1 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0},

        {'params': [p for n, p in param_optimizer2 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 1e-3,
         'lr': 1e-3},
        {'params': [p for n, p in param_optimizer2 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0,
         'lr':1e-3},

        {'params': [p for n, p in param_optimizer3 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 1e-3,
         'lr':4e-3},
        {'params': [p for n, p in param_optimizer3 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0,
         'lr':4e-3}
    ]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

0 bert.embeddings.word_embeddings.weight 	 True
1 bert.embeddings.position_embeddings.weight 	 True
2 bert.embeddings.token_type_embeddings.weight 	 True
3 bert.embeddings.LayerNorm.weight 	 True
4 bert.embeddings.LayerNorm.bias 	 True
5 bert.encoder.layer.0.attention.self.query.weight 	 True
6 bert.encoder.layer.0.attention.self.query.bias 	 True
7 bert.encoder.layer.0.attention.self.key.weight 	 True
8 bert.encoder.layer.0.attention.self.key.bias 	 True
9 bert.encoder.layer.0.attention.self.value.weight 	 True
10 bert.encoder.layer.0.attention.self.value.bias 	 True
11 bert.encoder.layer.0.attention.output.dense.weight 	 True
12 bert.encoder.layer.0.attention.output.dense.bias 	 True
13 bert.encoder.layer.0.attention.output.LayerNorm.weight 	 True
14 bert.encoder.layer.0.attention.output.LayerNorm.bias 	 True
15 bert.encoder.layer.0.intermediate.dense.weight 	 True
16 bert.encoder.layer.0.intermediate.dense.bias 	 True
17 bert.encoder.layer.0.output.dense.weight 	 True
18 bert.encode

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup, get_constant_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

In [ ]:
epochs = 10
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_steps/10),
    num_training_steps=total_steps
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm, trange
import torch.nn.functional as F
log_soft = F.log_softmax

In [ ]:
%%time
## Store the average loss after each epoch so we can plot them.
train_loss_values, valid_loss_values = [], []
PATH = './gabert_aug_crf_ner.pt'
f1_max = 0
f1_list = []
hist = {}

for epoch in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()


        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model.forward_custom(b_input_ids, b_input_mask, b_labels, token_type_ids=None)
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("\nAverage train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    train_loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    model.eval()
    predictions_f1 , true_labels_f1 = [], []
    eval_loss = 0
    for batch in valid_dataloader:

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model.forward_custom(b_input_ids, b_input_mask, b_labels, token_type_ids=None)

        eval_loss += outputs[0].mean().item()
        predict_labels = outputs[1]
        label_ids = b_labels.to('cpu').numpy().tolist()
        predictions = []
        for predict_label in predict_labels:
            predictions.append(predict_label)

        for b_input_id, preds, labels in zip(b_input_ids, predictions, label_ids):
            tokens = tokenizer.convert_ids_to_tokens(b_input_id.to('cpu').numpy())

            new_tokens, new_labels, new_preds = [], [], []
            for token, label_idx, pred in zip(tokens, labels, preds):
                if token.startswith("##"):
                    new_tokens[-1] = new_tokens[-1] + token[2:]
                else:
                    new_labels.append(label_idx)
                    new_preds.append(pred)
                    new_tokens.append(token)
            for token, pred, label in zip(new_tokens, new_preds, new_labels):
                predictions_f1.extend([pred])
                true_labels_f1.extend([label])
    eval_loss = eval_loss / len(valid_dataloader)
    print("Validation loss:", eval_loss)
    true_labels_f1_convert = [tag_values[:-1][i] for i in true_labels_f1]
    predictions_f1_convert = [tag_values[:-1][i] for i in predictions_f1]
    f1 = f1_score(true_labels_f1_convert, predictions_f1_convert,labels= tag_values[:-1] ,average='macro')
    print(classification_report(true_labels_f1_convert, predictions_f1_convert, labels= tag_values[:-1], digits=4))


    # eval_loss, f1 = evaluate(BIO_dataloader_valid,device)
    valid_loss_values.append(eval_loss)
    f1_list.append(f1)
    hist['train_loss_values'] = train_loss_values
    hist['valid_loss_values'] = valid_loss_values
    hist['f1_list'] = f1_list
    if f1 > f1_max:

        print(f'f1_score improved from: {f1_max:.4f} to {f1:.4f}')
        print(f'Best model saved to {PATH}')
        f1_max = f1
        patience = 5
        epochs_no_improve = 0
        best_epoch = epoch
    else:
        print(f'f1_score dont improve from: {f1_max:.4f} to {f1:.4f}')
        epochs_no_improve += 1
        if epochs_no_improve < patience:
            print(f'EarlyStopping count: {epochs_no_improve}/{patience}')
        else:
            print(f'\nEarly Stopping! Total epochs: {epochs}. Best epoch: {best_epoch} with f1_score: {f1_max:.4f}')
            break

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:519.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)



Average train loss: 40.975645273923874


Epoch:  10%|█         | 1/10 [01:01<09:15, 61.71s/it]

Validation loss: 7.470218777656555
              precision    recall  f1-score   support

           O     0.9733    0.9805    0.9769      2567
       B-PER     0.6981    0.5522    0.6167        67
       I-PER     0.8226    0.5730    0.6755        89
       B-LOC     0.7692    0.8065    0.7874        62
       I-LOC     0.8108    0.7895    0.8000        38
       B-ORG     0.7183    0.6296    0.6711        81
       I-ORG     0.5833    0.7549    0.6581       102

    accuracy                         0.9358      3006
   macro avg     0.7680    0.7266    0.7408      3006
weighted avg     0.9364    0.9358    0.9347      3006

f1_score improved from: 0.0000 to 0.7408
Best model saved to ./gabert_aug_crf_ner.pt

Average train loss: 9.436868280172348


Epoch:  20%|██        | 2/10 [01:54<07:31, 56.41s/it]

...

Validation loss: 8.208058953285217
              precision    recall  f1-score   support

           O     0.9902    0.9813    0.9857      2567
       B-PER     0.8548    0.7910    0.8217        67
       I-PER     0.8354    0.7416    0.7857        89
       B-LOC     0.9474    0.8710    0.9076        62
       I-LOC     0.8605    0.9737    0.9136        38
       B-ORG     0.7041    0.8519    0.7709        81
       I-ORG     0.6992    0.8431    0.7644       102

    accuracy                         0.9594      3006
   macro avg     0.8417    0.8648    0.8500      3006
weighted avg     0.9625    0.9594    0.9603      3006

f1_score improved from: 0.8475 to 0.8500
Best model saved to ./gabert_aug_crf_ner.pt

Average train loss: 0.2658078917302191


Epoch:  90%|█████████ | 9/10 [08:05<00:53, 53.45s/it]

Validation loss: 8.284355163574219
              precision    recall  f1-score   support

           O     0.9917    0.9801    0.9859      2567
       B-PER     0.8281    0.7910    0.8092        67
       I-PER     0.8293    0.7640    0.7953        89
       B-LOC     0.9636    0.8548    0.9060        62
       I-LOC     0.8409    0.9737    0.9024        38
       B-ORG     0.7113    0.8519    0.7753        81
       I-ORG     0.6850    0.8529    0.7598       102

    accuracy                         0.9591      3006
   macro avg     0.8357    0.8669    0.8477      3006
weighted avg     0.9628    0.9591    0.9603      3006

f1_score dont improve from: 0.8500 to 0.8477
EarlyStopping count: 1/5

Average train loss: 0.2181034639943391


Epoch: 100%|██████████| 10/10 [08:57<00:00, 53.76s/it]

Validation loss: 8.271434664726257
              precision    recall  f1-score   support

           O     0.9917    0.9801    0.9859      2567
       B-PER     0.8281    0.7910    0.8092        67
       I-PER     0.8293    0.7640    0.7953        89
       B-LOC     0.9636    0.8548    0.9060        62
       I-LOC     0.8409    0.9737    0.9024        38
       B-ORG     0.7083    0.8395    0.7684        81
       I-ORG     0.6797    0.8529    0.7565       102

    accuracy                         0.9587      3006
   macro avg     0.8345    0.8652    0.8462      3006
weighted avg     0.9626    0.9587    0.9600      3006

f1_score dont improve from: 0.8500 to 0.8462
EarlyStopping count: 2/5
CPU times: user 8min 32s, sys: 2.64 s, total: 8min 34s
Wall time: 8min 57s


In [ ]:
PATH = './gabert_aug_crf_ner.pt'
torch.save(model.state_dict(), PATH)

In [ ]:
model.load_state_dict(torch.load('./gabert_aug_crf_ner.pt'))

<All keys matched successfully>

In [ ]:
model.eval()

predicted_labels = []
true_labels = []

# Iterate through the test_dataloader to get the predictions
# Using the GPU
for batch in tst_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

    # Disabling gradient calculation for evaluation
    with torch.no_grad():
        #outputs = model(**inputs)
        outputs = model.forward_custom(batch[0], batch[1], batch[2], token_type_ids=None)

    # Extracting the predictions and also the true labels for the test data
    predict_labels = outputs[1]
    #print(predict_labels)
    predictions = []
    for predict_label in predict_labels:
      print(predict_label)
      predicted_labels.append(predict_label)
    #predicted_labels.append(predictions)

    true_labels.extend(inputs['labels'].tolist())

[0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 3, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 2, 2, 2, 2, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 5, 5, 5, 6, 0, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0]
[0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 6, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [ ]:
print(predicted_labels)
print(true_labels)

[[0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 3, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 2, 2, 2, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 5, 5, 5, 6, 0, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0], [0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 6, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
trues = []

for sentences in true_labels:
  t = []
  for vals in sentences:
    if vals != 7:
      t.append(vals)
  trues.append(t)

In [ ]:
preds=predicted_labels

In [ ]:
print(len(preds))

140


In [ ]:
print(test_tokenized_texts)
print(test_labels)
print(preds)

[['Mar', 'a', 't', '##ch', '##ítear', 'do', 'Sheos', '##amh', 'Mac', 'Grianna', 'é', 'caithfidh', 'an', 't', '-', 'ealaíontóir', 'an', 'solas', 'a', 'thabhairt', 'don', 'saol', 'agus', 'diúltú', 'do', 'chath', '##ú', 'sin', 'na', 'truaill', '##íochta', 'a', 'chuireann', 'an', 'saol', 'ina', 'chos', '##án', '.'], ['Dhiúltaigh', 'John', '##atha', '##n', 'Cape', 'agus', 'cúpla', 'foilsitheoir', 'eile', 'an', 'leabhar', 'a', 'fhoilsiú', 'agus', 'bhí', 'Nao', '##mi', 'le', 'ceangal', 'go', 'raibh', 'srian', 'a', 'chur', 'ar', 'a', 'cuid', 'tuairimíocht', '##a', '.'], ['Grianghraif', 'le', 'Maidhc', 'Ó', 'Seach', '##nas', '##aí', '.'], ['Tagann', 'a', 'ráiteas', 'tar', 'éis', 'don', 'chomhlacht', 'a', 'rá', 'le', 'hoibrithe', 'an', 'iarnróid', 'mí', 'ó', 'shin', 'go', 'raibh', 'sí', 'le', 'dúnadh', ':', "'", 'Chu', '##amar', 'ag', 'cruinniú', 'i', 'mBaile', 'Átha', 'an', 'Rí', 'agus', 'dúirt', 'an', 'bhainistíocht', 'linn', 'ansin', 'go', 'raibh', 'deireadh', 'leis', 'an', 'tseirbhís', 'last

In [ ]:
i = 0
while i < len(preds):
  if len(test_labels[i]) != len(preds[i]):
    print(f'issue: {i} {len(test_labels[i])}, {len(preds[i])}')
    print(f'{test_tokenized_texts[i]}\n{test_labels[i]}\n{preds[i]}')
    i += 1
  i += 1

In [ ]:
import pandas as pd

label_map = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-LOC',
    4: 'I-LOC',
    5: 'B-ORG',
    6: 'I-ORG'
}

# Initialise an empty list to store DataFrames
dfs = []

# Iterate through the tokens, labels, and predictions
for tokens, labels, predictions in zip(test_tokenized_texts, test_labels, preds):
    # Create a DataFrame from the current sublist
    temp_df = pd.DataFrame({'Word': tokens, 'POS': 'X', 'True': labels, 'Predicted': [label_map[pred] for pred in predictions]})
    # Append the DataFrame to the list
    dfs.append(temp_df)
    # Add an empty row as a DataFrame to the list
    dfs.append(pd.DataFrame({'Word': [''], 'POS': [''], 'True': [''], 'Predicted': ['']}))

# Concatenate the DataFrames along the rows axis
df = pd.concat(dfs, ignore_index=True)

print(df)

           Word POS   True Predicted
0           Mar   X      O         O
1             a   X      O         O
2             t   X      O         O
3          ##ch   X      O         O
4       ##ítear   X      O         O
...         ...  ..    ...       ...
4839          a   X      O         O
4840     bhfuil   X      O         O
4841  Gaeltacht   X  B-LOC     B-LOC
4842        ann   X      O         O
4843                                

[4844 rows x 4 columns]


In [ ]:
print(df.head(80))

       Word POS   True Predicted
0       Mar   X      O         O
1         a   X      O         O
2         t   X      O         O
3      ##ch   X      O         O
4   ##ítear   X      O         O
..      ...  ..    ...       ...
75    Seach   X  I-PER     I-PER
76    ##nas   X  I-PER     I-PER
77     ##aí   X  I-PER     I-PER
78        .   X      O         O
79                              

[80 rows x 4 columns]


In [ ]:
conll_format = ""

for index, row in df.iterrows():
    text = row['Word']
    pos = row['POS']
    tag = row['True']
    mapped_tag = row['Predicted']

    # Append the token in CoNLL format (word, gold_label, predicted_label)
    conll_format += f"{text}\t{pos}\t{tag}\t{mapped_tag}\n"

# Write the CoNLL format string to a text file
with open('/content/gaBERT_V2_5_eval.conll', 'w') as f:
    f.write(conll_format)